In [18]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('feature_selected.csv')

In [3]:
df.head()

,radius_mean_labels,perimeter_mean_labels,area_mean_labels,compactness_mean_labels,perimeter_se_labels,compactness_se_labels,radius_worst_labels,texture_worst_labels,perimeter_worst_labels,smoothness_worst_labels,diagnosis
0,2,2,1,3,1,1,3,0,3,3,M
1,3,3,2,0,0,0,3,1,2,1,M
2,3,2,2,2,0,1,2,1,2,2,M
3,1,1,0,4,0,2,1,1,1,4,M
4,3,3,2,1,1,0,2,0,2,2,M


In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['diagnosis'] = labelencoder.fit_transform(df['diagnosis'])

In [9]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

In [10]:
x = df.drop('diagnosis',axis='columns')
y = df['diagnosis']

In [11]:
clf = DecisionTreeClassifier(max_depth=3)
model = clf.fit(x, y)

In [12]:
print(df.columns)

Index(['radius_mean_labels', 'perimeter_mean_labels', 'area_mean_labels',
       'compactness_mean_labels', 'perimeter_se_labels',
       'compactness_se_labels', 'radius_worst_labels', 'texture_worst_labels',
       'perimeter_worst_labels', 'smoothness_worst_labels', 'diagnosis'],
      dtype='object')


In [13]:
feature_names_1=['radius_mean_labels', 'perimeter_mean_labels', 'area_mean_labels',
       'compactness_mean_labels', 'perimeter_se_labels',
       'compactness_se_labels', 'radius_worst_labels', 'texture_worst_labels',
       'perimeter_worst_labels', 'smoothness_worst_labels']

In [16]:
from sklearn.tree import _tree
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} "
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

In [19]:
rules = get_rules(clf,feature_names_1, '10')
for r in rules:
    print(r)

if (perimeter_mean_labels <= 1.5) and (perimeter_worst_labels <= 0.5) and (texture_worst_labels <= 2.5) then class: 1  | based on 208 samples
if (perimeter_mean_labels > 1.5) and (radius_worst_labels > 1.5) and (smoothness_worst_labels > 0.5) then class: 0  | based on 134 samples
if (perimeter_mean_labels <= 1.5) and (perimeter_worst_labels > 0.5) and (smoothness_worst_labels <= 1.5) then class: 1  | based on 114 samples
if (perimeter_mean_labels <= 1.5) and (perimeter_worst_labels > 0.5) and (smoothness_worst_labels > 1.5) then class: 0  | based on 74 samples
if (perimeter_mean_labels > 1.5) and (radius_worst_labels <= 1.5) and (compactness_se_labels > 0.5) then class: 0  | based on 14 samples
if (perimeter_mean_labels <= 1.5) and (perimeter_worst_labels <= 0.5) and (texture_worst_labels > 2.5) then class: 1  | based on 13 samples
if (perimeter_mean_labels > 1.5) and (radius_worst_labels <= 1.5) and (compactness_se_labels <= 0.5) then class: 1  | based on 9 samples
if (perimeter_mean_

In [22]:
l=['perimeter_mean_labels','perimeter_worst_labels','texture_worst_labels','radius_worst_labels','smoothness_worst_labels','compactness_se_labels','diagnosis']

In [23]:
new_f = df[l]

In [24]:
new_f.to_csv("rules.csv", index=False)